<a href="https://colab.research.google.com/github/dmkant/Camino/blob/master/Rendu_Intermediaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import numpy.random as r
import itertools as it
import pandas as pd
import time
import matplotlib.pyplot as plt
import math
!pip install pulp
from pulp import *

     |████████████████████████████████| 40.6MB 105kB/s 


<h1><center><b>Compte-Rendu Intermediaire pour un problème de PCCC</b></center></h1>

## **Bilan**
A ce stade du projet, nous nous sommes intéréssés à un problème de PCCC dans un cadre restrint (contraintes portant sur la borne superieure+contraintes finales). Nous avons mis en place une class afin de modéliser nos graphes et générer des exemples, écrit une fonction qui résout notre problème de PCCC avec l'algorithme à correction d'étiquette et effectuer une modélisatioin par PLNE.
### Objectif:
Pour la suite du Projet, nous allons nous intérrésser à un cadre plus large :
* contraintes à fenêtre de temps
* contraintes avec une borne supérieur et une borne inférieur

De plus nous allons aussi étudier la complexité de nos algorithmes.

## **Formalisation du problème**

Soit G = (V, A) un graphe orienté avec : 

*   |V | = n
*   |A| = m
	
Soit R un ensemble de ressources avec |R| = $n_r$.
A chaque arc (i,j) de A, on associe un cout c<sub>i,j</sub>  (on suppose positif ) et un vecteur de consommation de ressource t<sub>i,j</sub>   de dimension $n_r$  
On considère des contraintes de ressources finales portant uniquement sur la borne supérieure.

**Objectif:**
On cherche a trouver un chemin qui minimise le coût et dont la consommation de ressource (noté T) respecte les contraintes.









## Modélisation et génération du graphe
Nous avons décidé de modéliser nos graphes à l'aide du paradigme de la Programmation Orienté Objet(POO). Vous pouvez voir ci-dessous la classe *Graphe* dont chaque instance contient les attributs **ressource** (liste contenant l'ensemble des ressources),**nb_ressource**,  **noeud**, **nb_arc**  et **arc** (dictionnaire contenant l'ensemble des arcs avec leurs coût et leur consommation de ressource)


In [0]:
"""
    Modélisation d'un graphe : classe Graphe
    Les arcs du graphes seront modélisé sous la forme d'un double-dictionnaire 
    arc[S1 : sommet origine][S2 : sommet but ] = vecteur 
    Si le vecteur existe, alors il y a un arc entre S1 et S2
    Le vecteur sera de taille 1+R et représentera le vecteur cout et ressources de l'arc
    vecteur[0] = cout de l'arc
    vecteur[i] = Consommation ressource i sur l'arc ou i = [| 1 ; R |]
    
"""
class Graphe:
    nb_graph=0

    #res = nom des ressources à considérer
    def __init__(self,res=None):
        self.arc={} #double dictionnaire
        self.nb_arc=0
        self.noeud=[] #noeud
        if(type(res) is not list):
          res=[res]
        self.ressource=res # nom des ressources
        #Nombre de ressource
        if(res==[None]):
          self.nb_ressources = 0
        else:  
          self.nb_ressources = len(res) 
        Graphe.nb_graph+=1
        #Cas ou on veut verifer son caractère acyclique
        self.sigma = {} #ordre topologique
        self.couleur ={} #detection de circuit
    
    #Fonction permettant d'ajouter une/des ressources
    def ajout_ressource(self,nom_ress):
      if (type(nom_ress) is not list):
        nom_ress=[nom_ress]
      for nom in nom_ress:
        if(self.ressource==[None]):
          self.ressource=[nom]
        else: 
          self.ressource.append(nom)
        self.nb_ressources+=1
        for source in self.arc.keys():
          for cible in self.arc[source].keys():
           self.arc[source][cible]=np.append(self.arc[source][cible],0)
        
    #Fonction permettant l'ajout d'un/des sommet au graphe
    def ajoutNoeud(self,sommet):
      if (type(sommet) is not list):
        sommet=[sommet]
      sommet=[str(som) for som in sommet]
      for som in sommet:
        if (som in self.noeud):
          print('Le sommet ',som,' est déjà présent dans votre graph')
        else:
          self.noeud.append(som)
          self.couleur[som]=0
    
    #Fonction permettant l'ajout d'un/des arc entre 2 sommets
    def ajoutArc(self,origin,dest,conso):
        #conso = Cout de l'emprunt de l'arc et consommation des ressources
        if(type(origin) is not list):
          origin=[origin]
        origin=[str(ori) for ori in origin]
        if(type(dest) is not list):
          dest=[dest]
        dest=[str(des) for des in dest]
        if (type(conso) is not list):
          conso=[conso]
        if(type(conso[0]) is not list):
          conso=[conso]
        for i in range(len(origin)):
          if(len(conso[i])==self.nb_ressources+1):
            if(origin[i] not in self.noeud):
                self.ajoutNoeud(origin[i])
            if(origin[i] not in self.arc.keys()):
                self.arc[origin[i]]={}
            if (dest[i] not in self.noeud):
              self.ajoutNoeud(dest[i])
            if (dest[i] not in self.arc[origin[i]].keys()):
                self.arc[origin[i]][dest[i]]=np.array([conso[i]])
                self.nb_arc+=1
            else:
                present=any([all(conso[i]==self.arc[origin[i]][dest[i]][h]) for h in range(len(self.arc[origin[i]][dest[i]]))])
                if (not present):
                    self.arc[origin[i]][dest[i]]=np.append(self.arc[origin[i]][dest[i]],np.array([conso[i]]),axis=0)
                    self.nb_arc+=1
                else:
                    print(" l'arc (",origin[i],";",dest[i],") est déjà présent dans ce graphe avec le meme vecteur conso")
          else:
            print("Le vecteur conso de l'arc (",origin[i],dest[i],") n'a pas la bonne dimention. L'arc n'a donc pas été ajouté") 
        
    #Fonction retournant les predecesseurs d'un sommet sous forme de liste    
    def predecesseurs(self,sommet):
        sommet=str(sommet)
        liste_pred = []
        for key in self.arc.keys():
            if(sommet in self.arc[key].keys()):
                liste_pred.append(key)  
        return(liste_pred)
        
    #Fonction retournant les successeurs d'un sommet sous forme de liste
    def successeurs(self,sommet):
        sommet=str(sommet)
        liste_suc = []
        if(sommet in self.arc.keys()):
            for key in self.arc[sommet].keys():
                liste_suc.append(key)
        return(liste_suc)
    
    #Fonction affichant le graphe sous forme
    # Sommet(nombre de successeurs) : [{successeur_i : vecteur conso }, ... ]
    def affiche(self):
        for key in self.arc.keys():
            print(str(key)+"("+str(len(self.arc[key]))+"): [ ",end="")
            for cle in self.arc[key].keys():
                print("{"+str(cle)+" : "+str(self.arc[key][cle])+" } ,",end="")
            print("]")

    def colorier_noir(self,s,k):
        if(self.couleur[s]!=1):
            self.couleur[s]=1
            self.sigma[s] = k
        k+=-1
        for t in self.predecesseurs(s):
            if self.couleur[t]==2:
                if( all(self.couleur[i] == 1 for i in self.successeurs(t) )):
                    self.colorier_noir(t,k)
    
    def isAcyclic(self):
        acyclique = True
        k=len(self.noeud)
        PILE = []
        while(any(self.couleur[i] == 0 for i in self.noeud)):
            for i in self.noeud:
                if(self.couleur[i]==0):PILE=[i];break;
            while(len(PILE)!=0):
                traite = PILE[0]
                PILE.remove(traite)
                if(self.couleur[traite]==0):self.couleur[traite]=2;
                if(any(self.couleur[i]==2 for i in self.successeurs(traite))):
                    acyclique = False
                    for i in self.noeud:
                        self.couleur[i]=0
                    self.sigma ={}
                    print("Le graphe n'est pas acyclique")
                    return(acyclique)
                for i in self.successeurs(traite):
                    if(self.couleur[i]==0):PILE.insert(0,i);
                if(all(self.couleur[i]==1 for i in self.successeurs(traite)) or len(self.successeurs(traite))==0):
                    self.colorier_noir(traite, k)
                    k = min(self.sigma[k] for k in self.sigma.keys())-1
        for i in self.noeud:
            self.couleur[i]=0
        return(acyclique)

In [0]:
#Exemple d'utilisation
g=Graphe()
g.ajout_ressource("fatigue")
g.ajout_ressource(["prix essence","cris des enfants"])
g.ajoutNoeud("Paris")
g.ajoutNoeud(["Berlin","Tokyo"])
g.ajoutArc("Paris","Berlin",[1,2,3,4])
g.ajoutArc(["Berlin","Tokyo"],["Tokyo","Tokyo"],[[1,2,3,4],[4,3,2,1]])
g.affiche()
#--------------------------------------------------------------------------#
Paul = Graphe()
Paul.ajout_ressource(["nb_colab","distance","energie"])
Paul.ajoutNoeud(["Debut","Meetings","Conferences","Financement","Corruption","Demagogie","Election","Decheance","lol"])
Paul.ajoutArc(["Debut","Debut","Debut","Financement","Conferences","Conferences","Meetings","Corruption","Corruption","Demagogie","Election","lol"],
              ["Meetings","Conferences","Financement","Conferences","Corruption","Demagogie","Corruption","Election","Decheance","Election","Decheance","Debut"],
              [[15,1,3,4],[10,1,4,5],[8,2,6,5],[4,5,6,2],[20,2,0,1],[2,3,9,1],[10,3,0,4],[1,6,2,8],[2,6,6,4],[4,3,2,2],[4,5,6,4],[5,6,7,8]])
print(Paul.isAcyclic())
print(Paul.sigma)


Paris(1): [ {Berlin : [[1 2 3 4]] } ,]
Berlin(1): [ {Tokyo : [[1 2 3 4]] } ,]
Tokyo(1): [ {Tokyo : [[4 3 2 1]] } ,]
True
{'Decheance': 9, 'Election': 8, 'Demagogie': 7, 'Corruption': 6, 'Conferences': 5, 'Financement': 4, 'Meetings': 3, 'Debut': 2, 'lol': 1}


## Résolution d'un problème de Plus Court Chemin avec Contraintes(PCCC)
#### **Definition**

* **Etiquette** : vecteur associé à un chemin et dont les coordonnées sont le coût et les consommations des différentes ressources.
* **Orde de Pareto** : Une étiquette **E** domine **E'** ssi toute les cordonnées de E sont supérieur ou égale à celle de E' et qu’au moins une coordonnée de E est strictement supérieur à celle de E'.


#### **Algorithme à correction d'étiquettes**
On utilisera **l'algorithme à correction d'étiquettes**. 
On considère un graphe de source le sommet s et de puit le sommet t.
A chaque sommet i on associe un ensemble d'étiquettes **E<sub>s,i</sub>** qui correspondes à des chemins entre la source et ce sommet.
A l'initialisation, tous les ensembles **E<sub>s,i</sub>** sont vides car aucun chemin n'a été traité exépté **E<sub>s,s</sub>** qui vaut {0}.


On condisere une liste **L** qui contient l'ensemble des sommet dont au moins une étiquette n'a pas encore été propagé. On met a jour **L** a chaque itération.
  
A chaque intération, on choisi un sommet de **L** et on propage ses étiquette à ses successeurs si cette propagation respecte les contraintes. On élimine ce sommet de **L**. On ajoute ainsi de nouvelles étiquettes à ses successeur sur lesquelles on effectuera un test de domination au sens de l'ordre de Pareto afin de ne conserver que les étiquettes non-dominées.
Ce test assure que les **E<sub>s,i<sub>** sont Pareto-optimal et permet d'éventuellement **corriger** les étiquettes déjà existantes , d'où le nom de l'algorithme.

A chaque fois qu'on crée une nouvelle étiquette qui est maintenu après le test de domination, le sommet qui lui correspond est ajouté à la liste **L**. L'algorithme s'arrête une fois que la liste est vide donc lorsque tous les sommets sont traités et que les propagations ne change plus les étiquettes.

Sur tous les sommets et particulièrement en t, l'algorithme permet d'obtenir une ou plusieurs étiquettes non dominées  donc candidates comme solutions du problème.
Etant donné que le but est de minimiser le cout de s à t, l'algorithme renvoie le chemin ayant le cout le plus faible en t. S'il y en a plusieurs, on en choisit un. 
Notons que dans cet algorithme, on calcule le plus court chemin contraint de s à tous les autres sommets du graphe.

![Algorithme](https://drive.google.com/uc?id=1GNPnNF0-vO_jykku-rCmQeSDKuthGv9E)


In [0]:
""" Création d'une fonction Pareto qui prend comme paramètre un ensemble d'étiquette 
(et argument facultatif correspondant aux prédécésseur de ces etiquette) et renvoie
un ensemble Pareto minimale
"""
def ordre_Pareto(arc1,arc2):
    arc = arc1 - arc2
    retour = None
    if np.all(arc>=0)==True:
        retour =np.array([arc2])
    elif np.all(arc<=0)==True:
        retour = np.array([arc1.tolist()])
    else:
        retour = np.array([arc1,arc2])
    return(retour)

#Fonction qui permet de supprimer les doublons d'un ensemble d'étiquette avec leur prédécesseurs
def supp_doublon(E,P):
  #enleve les valeurs dupliques
  uniq_value=np.unique(E,axis=0,return_index=True,return_counts=True)
  if(P is not None):
    P=P[uniq_value[1]]
  E=uniq_value[0]
  return([E,P])

#Fonction renvoie l'ensemble Pareto optimal d'un ensemble d'étiquette avec leurs prédécesseur 
def Pareto(Etiq,Pred=None):
   #Enleve les étiquettes dupliqués
   [Etiq,Pred]=supp_doublon(Etiq,Pred)
   #Conserve uniquement les element non dominés
   i=0
   while(i < len(Etiq)): 
     element1=Etiq[i]
     j=0
     while(j < len(Etiq)):
       if(i!=j):
        element2=Etiq[j]
        test=ordre_Pareto(element1,element2)
        if(len(test)==1):
          supp=j if np.all(element1==test[0]) else i
          Etiq=np.delete(Etiq,supp,axis=0)
          if(Pred is not None):
            Pred=np.delete(Pred,supp)
       j+=1 
     i+=1
   resu=[Pred,Etiq] if Pred is not None else Etiq
   return(resu) 

In [0]:
# Exemple d'utilisation
E=np.array([[30,15,11,20],
            [27,10,9,12],
            [30,15,11,20],
            [32,9,10,10],
            [35,14,11,18]])
P=np.array(["A","B","C","D","E"])

Pareto(E,P)


[array(['B', 'D'], dtype='<U1'), array([[27, 10,  9, 12],
        [32,  9, 10, 10]])]

In [0]:
""" Algorithme à correction d'étiquettes 
"""

#Fonction qui renvoie le plus court chemin dans l'algorithme à correstion d'etiquette
def PCC(E,source="s",puit="t"):
  chemin=[puit]
  if E[puit]==[]:
      print("Aucun chemin respectant les contraintes ")
      return(-1)
  else:
      #etiquette du puit correspondant au chemin qui minimise le cout
      cout_mini_chemin=E[puit][1][::,0]==min(E[puit][1][::,0])
      pred=E[puit][0][cout_mini_chemin][0]
      chemin.append(pred)
      #remonte les etiquettes du puit jusqu'a la source pour construire le chemin
      while pred!=source:
        cout_mini=E[pred][1][::,0]==min(E[pred][1][::,0])
        pred=E[pred][0][cout_mini][0]
        chemin.append(pred)
      return([chemin[::-1],E[puit][1][cout_mini_chemin]])

#Fonction correspondant a l'algorithme a correction d'etiquette
def correc_etiquette(g,contraintes,source="s",puit="t"):
  if(source not in g.noeud or puit not in g.noeud):
    print("Erreur: votre source ou votre puit ne figurent pas parmi les sommets de votre graph")
    return(-1)
  #Initialisation
  List=[source]
  Etiq={noeud: [] for noeud in g.noeud}
  Etiq[source]=[np.array([source]),np.array([np.zeros(g.nb_ressources+1)])]
  itera=0
  temps_pareto=0
  while(len(List)!=0 and itera<=50000):
    itera+=1
    #print(itera)
    i=r.choice(List)
    List.remove(i)
    if(i in g.arc.keys()):
      for j in g.successeurs(i):
        for k in range(len(Etiq[i][1])):
          E=Etiq[i][1][k]
          arc_ij=Pareto(g.arc[i][j])
          for h in range(len(arc_ij)):
              if(np.all(arc_ij[h][1:len(arc_ij[h])]+E[1:len(E)]<=contraintes)):
                E2=arc_ij[h]+E
                ajout=True
                if len(Etiq[j])==0:
                  Etiq[j]=[np.array([i]),np.array([E2])]
                else:
                  Etiq[j][1]=np.append(Etiq[j][1],np.array([E2]),axis=0)
                  Etiq[j][0]=np.append(Etiq[j][0],i)
                t1=time.time()
                Etiq[j]=Pareto(Etiq[j][1],Etiq[j][0])
                temps_pareto+=(time.time()-t1)
                if(E2 in Etiq[j][1] and ajout):
                  List.append(j)
                  List=list(np.unique(List))
              #else:
                  #print("Contraintes non respecte")

  explosion=True if itera>=50000 else False
               
  return([PCC(Etiq,source=source,puit=puit),explosion,temps_pareto])



#### **Exemple de résolution**

Un jeune politicien, Paul Politics, a pour objectif de se présenter aux prochaines élections. Sauf surprise, il est assuré de gagner. Cependant, il sait également que la vie d'un politicien classique se termine lorsqu'il finit "déchu", n'ayant plus le soutien de la population. 
Entre le début de la campagne et sa certaine déchéance, son objectif est de minimiser l'argent investi tout en ayant des contraintes sur 3 ressources : le nombre de ses collaborateurs, son énergie dépensée (son niveau d'implication), la distance parcourue en déplacement.
Entre le debut de sa campagne et sa decheance, il peut passer par certaines étapes qui seront les sommets de notre graphe : la source sera donc le debut de la campagne et le puit la decheance.



In [0]:
Paul = Graphe()
Paul.ajout_ressource(["nb_colab","distance","energie"])
Paul.ajoutNoeud(["Debut","Meetings","Conferences","Financement","Corruption","Demagogie","Election","Decheance"])
Paul.ajoutArc(["Debut","Debut","Debut","Financement","Conferences","Conferences","Meetings","Corruption","Corruption","Demagogie","Election"],
              ["Meetings","Conferences","Financement","Conferences","Corruption","Demagogie","Corruption","Election","Decheance","Election","Decheance"],
              [[15,1,3,4],[10,1,4,5],[8,2,6,5],[4,5,6,2],[20,2,0,1],[2,3,9,1],[10,3,0,4],[1,6,2,8],[2,6,6,4],[4,3,2,2],[4,5,6,4]])

"""Vecteur consommation max de ressources 
1 - Pas plus de 12  collaborateurs
2-  Pas plus de 20 unités d'energie
3-  Une distance inférieure à 15 unités"""
conso = np.array([12,20,15])
parcours = correc_etiquette(Paul,conso,"Debut","Decheance")
print(parcours)

[[['Debut', 'Meetings', 'Corruption', 'Decheance'], array([[27., 10.,  9., 12.]])], False, 0.0019524097442626953]


# Résolution par PuLP du modèle de PLNE


Ici, on s'intéressera à la modélisation en PLNE du problème de PCCC (Plus Court Chemin avec Contraintes de ressources). Le problème a été formalisé en première partie du notebook. On cherche le plus court chemin de s à t en respectant les contraintes de ressources.


**Variables**  
Pour tout $a \in A$ : $x_{a} \in ${$0,1$}$ = 1 $ si et seulement si $a$ est dans le chemin choisi  

**Objectif**  
Minimiser le coût du chemin:  
$ {min} \sum \limits_{{a \in A}} c_a*x_a $

**Contraintes**  
Conservation du flot:  $ \forall i \ne s,t $ on a  $ \sum \limits_{(i,j)\in \delta^+(i)} x_{ij} = \sum \limits_{(j,i)\in \delta^-(i)} x_{ji}   $

Flot entrant de t:  $ \sum \limits_{(i,t) \in \delta^-(t)} x_{it} = 1 $  
Flot sortant de s:  $ \sum \limits_{(s,i) \in \delta^+(s)} x_{si} = 1 $  
Contraintes de ressources sur les chemins:
$ \forall r\in ${$1,2,\dots, n_r $}, $\sum \limits_{a \in A} t^r_a*x_a \le b^r$ où $b^r$ est la consommation maximale de la ressource $r$.






In [0]:
def Consommation(graphe):
  n_graphe=[[len(graphe.arc[key1][key2]) for key2 in graphe.arc[key1].keys()]for key1 in graphe.arc.keys()]
  print(n_graphe)
  n_graphe=np.max(np.max(n_graphe))
  print(n_graphe)
  Conso={e:{} for e in ["couts"]+["r"+str(i+1) for i in range(graphe.nb_ressources)]}
  #Recupération des coûts associés à chaque arc, cout nul si arc non existant:
  for key1 in graphe.noeud:
    Conso['couts'][key1]={}
    for key2 in graphe.noeud:
      Conso['couts'][key1][key2]={}
      for n in range(n_graphe):
        if key1 in graphe.arc.keys():
          if key2 in graphe.arc[key1].keys():
            if n < len(graphe.arc[key1][key2]):
              Conso['couts'][key1][key2][n]=graphe.arc[key1][key2][n][0]
            else:
              Conso['couts'][key1][key2][n]=0
          else:
            Conso['couts'][key1][key2][n]=0
        else:
          Conso['couts'][key1][key2][n]=0
  #Pareil pour les ressources
  for e in range(graphe.nb_ressources):
    for key1 in graphe.noeud:
      Conso["r"+str(e+1)][key1]={}
      for key2 in graphe.noeud:
        Conso["r"+str(e+1)][key1][key2]={}
        for n in range(n_graphe):
          if key1 in graphe.arc.keys():
            if key2 in graphe.arc[key1].keys():
              if n < len(graphe.arc[key1][key2]):
                Conso["r"+str(e+1)][key1][key2][n]=graphe.arc[key1][key2][n][e+1]
              else:
                Conso["r"+str(e+1)][key1][key2][n]=0
            else:
              Conso["r"+str(e+1)][key1][key2][n]=0
          else:
            Conso["r"+str(e+1)][key1][key2][n]=0
  return([Conso,n_graphe])
  
def PCC_PLNE(graphe,b,source="s",puit="t"):
  Conso=Consommation(graphe)
  n_graphe=Conso[1]
  Conso=Conso[0]
  print(graphe.noeud)
  #Modelisation du probleme
  prob = LpProblem("PLNE", LpMinimize)
  ############################Variables#################################
  lignes = [e for e in graphe.noeud]
  x=LpVariable.dicts("x",(lignes,lignes,[i for i in range(n_graphe)]), 0,1,LpInteger)
  
  ############################Objectif##################################
  prob+=lpSum([lpSum([lpSum([x[i][j][k]*Conso['couts'][i][j][k] for k in range(n_graphe)]) for i in graphe.noeud]) for j in graphe.noeud])

  ###########################Contraintes################################ 
  #Conservation du flot
  for i in [e for e in graphe.noeud if e!=source and e!=puit ]:
    prob+=lpSum([lpSum([x[i][j][k] for k in range(n_graphe)]) for j in graphe.successeurs(i)])==lpSum([lpSum([x[j][i][k] for k in range(n_graphe)])  for j in graphe.predecesseurs(i)])

  #Flot sortant de la source et flot sortant du puit égal à 1
  prob+=lpSum([lpSum([x[source][j][k] for k in range(n_graphe)])  for j in graphe.successeurs(source)])==1
  prob+=lpSum([lpSum([x[j][puit][k] for k in range(n_graphe)])  for j in graphe.predecesseurs(puit)])==1

  #Si un arc 'a' n'existe pas, x[a] = 0
  for i in graphe.noeud:
    if i not in graphe.arc.keys():
      for j in graphe.noeud:
        for k in range(n_graphe):
          prob+=x[i][j][k]==0
    else:
      for j in graphe.noeud:
        for k in range(n_graphe):
          if j not in graphe.arc[i].keys():
            prob+=x[i][j][k]==0
          else:
            if k >= len(graphe.arc[i][j]):
              prob+=x[i][j][k]==0

  #Contraintes de ressources finales
  for e in range(graphe.nb_ressources):
    #Ressource
    prob+=lpSum([lpSum([lpSum([x[key1][key2][k]*Conso["r"+str(e+1)][key1][key2][k] for k in range(n_graphe)]) for key1 in graphe.noeud]) for key2 in  graphe.noeud]) <=b[e]


  ######################### Resolution ###########################################

  #Affichage du modèle
  #prob.writeLP("furniture.lp")
  #print(prob)
  #Résolution du modèle de PLNE
  prob.solve()
  #print("Status:", LpStatus[prob.status])
  # On affiche le chemin
  print("-------------------------------------------------------------")
  #print("Affichage des résultats")
  #for v in prob.variables():
      #if(v.varValue!=0):
        #print(v.name, "=", v.varValue)
  # The optimised objective function value is printed to the screen    
  #print("Cout minimal = ", value(prob.objective))

In [0]:
"""
On effectue un test avec 2 ressources et un graphe quelconque"""
graphe = Graphe()
graphe.ajout_ressource(["r1","r2"])
graphe.ajoutNoeud([0,1,2,3,4,5])
graphe.ajoutArc([0,0,1,1,2,2,3,3,4,4],
                [1,2,4,3,1,3,4,5,2,5],
                [[7,6,5],[4,3,2],[6,11,5],[8,10,4],[7,3,3],[9,4,2],[6,7,2],[7,5,4],[3,2,1],[3,4,3]])

"""Ici, la source sera le sommet 0 et 5 le sommet puit"""
b = [24,12]
PCC_PLNE(graphe,b,"0","5")

[[1, 1], [1, 1], [1, 1], [1, 1], [1, 1]]
1
['0', '1', '2', '3', '4', '5']
-------------------------------------------------------------


In [0]:
###############################################################################
#################Etude empirique PLNE##########################################
###############################################################################
"""n=[i for i in range(10,500,10)]
m=[2]
nb_res=[1]
b=[10]
C=[900]
resu=[list(tup) for tup in it.product(n,m,nb_res,b,C)]
for i in range(len(resu)):
  resu[i][1]=int(resu[i][1]*resu[i][0])
compar=pd.DataFrame(resu,columns = ['n', 'm', 'Ressource', 'b','Contraintes'])
compar['tempsPLNE']=None
compar['tempsPCCC']=None"""
###########################debut##############################################
for i in range(41,compar.shape[0],1):
        print("==========ITERATION========== :",i)
        temps1=[]
        temps2=[]
        for j in range(3):
            G=generateur_graph(compar['n'][i]-2,compar['m'][i],compar['Ressource'][i],b=compar['b'][i])
            #Calcul temps PLNE
            start_time = time.time()
            resu=PCC_PLNE(G,[compar['Contraintes'][i] for h in range(compar['Ressource'][i])],source="s",puit="t")
            temps1.append(time.time() - start_time)
            #Calcul temps correction d'étiquette
            start_time = time.time()
            resu=correc_etiquette(G,[compar['Contraintes'][i] for h in range(compar['Ressource'][i])],source="s",puit="t")
            temps2.append(time.time() - start_time)
        #Stockage des durees
        compar['tempsPLNE'][i]=np.mean(temps1)
        compar['tempsPCCC'][i]=np.mean(temps2)


compar.to_csv("Temps_plne.csv",sep =";",index = False)



==========ITERATION========== : 41
[[1, 1], [1, 1], [1, 1, 1], [1], [1, 1, 1], [1], [1, 1, 1], [1, 1], [1, 1, 1], [1, 1], [1], [1], [1, 1, 1, 1], [1], [1], [1, 1], [1, 1, 1], [1, 1, 1], [1], [1, 1], [1], [1, 1], [1], [1, 1], [1], [1, 1], [1], [1], [1, 1], [1], [1, 1, 1], [1], [1], [1, 1, 1, 1], [1, 1], [1], [1, 1], [1, 1, 1], [1], [1], [1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1], [1], [1, 1], [1, 1], [1, 1], [1], [1, 1, 1], [1, 1], [1, 1], [1], [1, 1, 1, 1, 1], [1], [1, 1, 1], [1, 1], [1, 1], [1, 1], [1], [1, 1, 1, 1, 1], [1], [1, 1], [1, 1], [1, 1], [1], [1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1], [1, 1, 1], [1, 1], [1], [1, 1], [1, 1, 1], [1, 1], [1, 1], [1], [1], [1, 1], [1, 1], [1, 1], [1, 1, 1, 1], [1, 1, 1], [1], [1, 1], [1], [1, 1], [1], [1, 1], [1, 1], [1, 1], [1], [1, 1, 1, 1], [1], [1, 1], [1, 1], [1, 1, 1], [1, 1], [1, 1], [1, 1, 1, 1], [1, 1, 1], [1], [1, 1], [1], [1, 1, 1, 1], [1, 1, 1], [1, 1, 1, 1], [1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1], [1], [1], [1, 1], [1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


==========ITERATION========== : 42
[[1, 1], [1, 1], [1, 1], [1, 1, 1], [1, 1, 1, 1], [1, 1, 1], [1], [1, 1, 1], [1, 1], [1], [1], [1], [1, 1], [1, 1], [1], [1], [1], [1, 1], [1], [1, 1], [1, 1], [1, 1], [1, 1], [1], [1, 1], [1, 1, 1], [1, 1, 1, 1], [1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1, 1], [1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1], [1], [1, 1], [1, 1], [1], [1], [1], [1, 1, 1], [1], [1], [1], [1], [1], [1, 1], [1, 1], [1, 1], [1, 1, 1, 1], [1, 1], [1], [1, 1], [1, 1], [1, 1, 1, 1], [1], [1, 1, 1], [1], [1, 1], [1, 1], [1, 1, 1], [1, 1], [1], [1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1, 1], [1], [1, 1], [1], [1], [1, 1], [1], [1, 1, 1], [1], [1, 1, 1], [1], [1, 1, 1], [1, 1], [1, 1, 1, 1], [1, 1, 1], [1, 1], [1, 1], [1], [1, 1, 1], [1, 1, 1], [1, 1], [1, 1, 1], [1], [1], [1], [1, 1], [1, 1, 1], [1, 1], [1], [1], [1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1], [1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1], [1], [1], [1, 1], [1], [1, 1], [1], [1, 1], [1, 1, 1], [1], [1], [

In [0]:
from google.colab import files
files.download('Temps_plne.csv')

In [0]:
compar[::][0:compar.shape[0]]

In [0]:

def graphe_alea(nb_sommets,proba=0.5): #proba = probabilité de generer un arc
    graphe=Graphe(res=["R1","R2"])
    graphe.ajoutNoeud([i for i in range(nb_sommets+1)])
    source=0;puit=nb_sommets
    if(proba<=0 or proba>=1):
        print("La proba doit être strictement comprise entre 0 et 1")
    else:
        #Creation chemin entre source et puit
        s1=source
        s2=r.randint(source+1,nb_sommets)
        conso = [r.randint(0,10),r.randint(0,20),r.randint(0,20)]
        graphe.ajoutArc(s1,s2,conso)
        while(s2!=puit):
          s1=s2
          s2=r.randint(source+1,nb_sommets)
          if(s1!=s2): #On evite les boucles sur les sommets
            conso = [r.randint(0,10),r.randint(0,20),r.randint(0,20)]
            graphe.ajoutArc(s1,s2,conso)
          else:
            s2+=1
            conso = [r.randint(0,10),r.randint(0,20),r.randint(0,20)]
            graphe.ajoutArc(s1,s2,conso)
          if(s2==puit):
            conso = [r.randint(0,10),r.randint(0,20),r.randint(0,20)]
            graphe.ajoutArc(s1,s2,conso)
        #Generation d'arcs aleatoires entre les différents sommets
        """Inspire de fast_gnp_random_graph de Networkx"""
        lp = math.log(1-proba)
        s1=source
        while(s1!=puit):
          s2 = r.randint(s1,puit)
          if(s1==s2):
            s2+=1
          lr=math.log(1-r.random())
          if(lr>=lp):
            if(s1!=s2):
              conso = [r.randint(0,10),r.randint(0,20),r.randint(0,20)]
              graphe.ajoutArc(s1,s2,conso)
          else:
            s1+=1
    return(graphe)



In [0]:
#Exemple d'utilisation
G1=graphe_alea(5)
G1.affiche()
print(G1.noeud)
pk = correc_etiquette(G1,np.array([50,50]),"0","5")
print(pk)
PCC_PLNE(G1,np.array([50,50]),"0","5")


### **Generateur de graphe par niveau**

Afin de rentrer dans le cadre d'un problème de PCCC, les graphes que nous générons doivent remplir certains critères. 

  * Tout d'abord, la source n'a aucun prédécésseur et le puit n'a aucun succésseur. 
  * De plus, chaque sommet doit avoir au moins un successeur et un prédécésseur afin de pouvoir potentiellement faire partie de la solution.
  * Il doit exister un chemin entre la source et chaque sommet dont, le plus important, la source 

La fonction *generateur_graph* ci dessous permet de générer des graphes remplissant les critères ci dessus et prends en paramètre:
  * **n** le nombre de sommets en dehors de la source et du puit
  * **m** : le nombre d'arcs du graph
  * **nb_ress**: le nombre de ressources
  * **a** : borne inf lors du tirage aleatoir des coûts et ressources des arcs
  * **b=10** : borne sup lors du tirage aleatoir des coûts et ressources des arcs
  * **graph_simple**: Booléan permettant de construire des graphes simples

Cette fonction permet de générer des graphes en plusieurs étapes.
### Première étape : Division
Elle crée un graph avec **n**+2 sommets numeroté de 0 à **n**+1 avec **nb_ress** ressources. Puis elle va répartir ces sommet en niveaux afin de garantir la creation d'un chemin entre la source et le puit. Le premier niveau comporte un seul sommet : la source; le dernier niveau comporte un seul niveau: le puit. Le nombre de niveau entre la source et le sommet est tiré aleatoirement ainsi que le nombre de sommet par niveau avec bien sur la contrainte que la somme des sommets vaut **n** (hors puit et source).
### Deuxième étape : Liaison Niveau
La fonction choisi deux sommets par niveau ( hormis le premier et dernier niveau qui ne comporte que la source et le puit) qui seriviront à faire la **liaison inter-niveau**: un pour la liaison entre son niveau et le précédent, un pour la liaison entre son niveau et le suivant.  Puis si, le sommet qui fait la liaison entre son niveau et le suivant n'est pas le même que celui qui fait la liaison  entre son niveau et le précédent, la fonction choisi un autre sommet de ce niveau afin de faire la **liaion intra-niveau**. Ainsi à l'issu de cette étape il existe un chemin entre la source et le puit. Puis, on lie les sommets n'ayant pas servies aux liaisons entre eux de manière à ce que chacun ait un successeur et un prédécesseur. Ainsi, à l'issu de cette étape, notre graphe comporte exactement **n**+1 sommets ou **n**+2 sommets

### Troisième étape : Completer le nombre d'arc
La dernière étape consiste à ajouter des arcs entre sommet aléatoire afin que notre graph ait m arcs


In [0]:

def division(n,m):
    #Nombre de niveau
    nb_niv=r.randint(1,n) if m!=(n+1) else n
    #Nombre de noeud par niveau
    nb_nd=[0]
    for i in range(nb_niv):
        max=n-np.sum(nb_nd)-(nb_niv-i-1)
        nb_nd.append(r.randint(1,max))
    #Contrainte sur le degré de liberte du nombre de noeud par niveau
    ind_cont=r.randint(1,nb_niv+1)
    nb_nd[ind_cont]=n-np.sum([j for i, j in enumerate(nb_nd) if i!=ind_cont])
    #construction des noeud
    nb_nd=np.cumsum(nb_nd)
    resu={j+1:[i+1 for i in range(nb_nd[j],nb_nd[j+1])] for j in range(len(nb_nd)-1)}
    resu[0]=["s"]
    resu[nb_niv+1]=["t"]
    return(resu)


def generateur_graph(n,m,nb_ress=1,a=1,b=10,graph_simple=True):
    if m<(n+1):
        print("Vous devez au moins entrez ",n+1, " arcs")
        return(-1)
    #Initialisation du graph    
    g=Graphe([i for i in range(1,nb_ress+1)])
    g.ajoutNoeud(['s','t'])
    g.ajoutNoeud([i for i in range(1,n+1)])
    
    #Creation des niveau    
    Niveau=division(n,m)
    nb_niv=len(Niveau)
    
    #Liaison niveau
    Inter_Niveau_P={'noeud':[r.choice(Niveau[x]) for x in range(0,nb_niv-1)],
       'ind':[r.choice(Niveau[x]) for x in range(1,nb_niv)]}
    
    Chek={i:{"P":None,"S":None} for i in range(n+2)}
    
    for i in range(0,nb_niv-1):
        #Liaison inter niveau
        #Pred
        noeud=Inter_Niveau_P['noeud'][i]
        ind=Inter_Niveau_P['ind'][i] if i!=(nb_niv-2) else (n+1)
        Chek[ind]['P']=noeud
        #Succ
        ind2=noeud if noeud!='s' else 0
        noeud2=ind if ind!=(n+1) else 't'
        Chek[ind2]['S']=noeud2
        #ajout de l'arc
        g.ajoutArc(noeud,noeud2,[r.randint(a,b) for h in range(nb_ress+1)])
        #Liaison intra niveau
        if i!=(nb_niv-2):
           if Inter_Niveau_P['ind'][i]!=Inter_Niveau_P['noeud'][i+1]:
               ind3=Inter_Niveau_P['noeud'][i+1]
               noeud3=Inter_Niveau_P['ind'][i]
               Chek[ind3]['P']=noeud3
               Chek[noeud3]['S']=ind3
               #ajout de l'arc
               g.ajoutArc(noeud3,ind3,[r.randint(a,b) for h in range(nb_ress+1)])
    
    #Associer à chaque noeud un predesseceur et un successeur
    sans_succ1=[i for i in Chek.keys() if Chek[i]['S'] is None and i!=n+1]
    if sans_succ1!=[]:
        debut=[i for i in Chek.keys() if Chek[i]['S'] is not None and i not in[0,n+1]]+['s']
        debut=r.choice(debut)
        fin=[i for i in Chek.keys() if Chek[i]['S'] is not None and i not in[0,n+1]]+['t']
        fin=r.choice(fin)
        nouveau_arc=[debut]+sans_succ1+[fin]
        nv_arc_debut=[nouveau_arc[i] for i in range(len(nouveau_arc)-1)]
        nv_arc_fin=[nouveau_arc[i] for i in range(1,len(nouveau_arc))]
        nv_et=[[r.randint(a,b) for h in range(nb_ress+1)] for i in range(len(nouveau_arc))]
        g.ajoutArc(nv_arc_debut,
                   nv_arc_fin,
                   nv_et)
        
    #Ajouter les arcs manquant
    nb_arc=m-g.nb_arc    
    arc_or=[k for k in g.noeud if k!='t']
    if not graph_simple:
        arc_dest=[k for k in g.noeud if k!='s']
        while nb_arc!=0:
            nv_arc_debut,nv_arc_fin=[r.choice(arc_or) for k in range(nb_arc)],[r.choice(arc_dest) for k in range(nb_arc)]
            nv_et=[[r.randint(a,b) for h in range(nb_ress+1)] for i in range(nb_arc)]
            g.ajoutArc(nv_arc_debut,
                       nv_arc_fin,
                       nv_et) 
            nb_arc=m-g.nb_arc 
    else:
        for i in range(nb_arc):
            arc_or2=r.choice(arc_or)
            arc_dest=[k for k in g.noeud if k!='s' and k not in g.successeurs(arc_or2)]
            arc_dest2=r.choice(arc_dest)
            g.ajoutArc(arc_or2,arc_dest2,[r.randint(a,b) for h in range(nb_ress+1)])
    
    return(g)    


In [0]:
g=generateur_graph(5, 7,nb_ress=4,graph_simple=True)
g.affiche()
g=generateur_graph(5, 10,a=15,b=20,graph_simple=False)
g.affiche()

In [0]:
"COMPARAISON DES GENERATEURS DE GRAPHES"

n=[i for i in range(50,501,50)]
temps1,temps2=[],[]
for i in range(len(n)):
    print("==========ITERATION========== :",i)
    moy1,moy2,moy3=[],[],[]
    for k in range(5):
        start_time = time.time()
        G1 = graphe_alea(n[i])
        moy1+=[time.time()-start_time]
        print("sommet",n[i],"arc", G1.nb_arc)
        start_time = time.time()
        G2=generateur_graph(n[i], G1.nb_arc,graph_simple=False)
        moy2+=[time.time()-start_time]
    temps1+=[np.sum(np.array(moy1))/5]
    temps2+=[np.sum(np.array(moy2))/5]

In [0]:
print(len(G1.noeud),G1.nb_arc)

In [0]:
#Representation graphique
#Trace 
plt.plot(n,temps1,n,temps2,n,temps3)
plt.title("Temps d execution en fonction de la taille du graphe genere aleatoirement")
plt.legend(["graphe_alea","generateur_graphe b=10","generateur_graphe b=100"])
plt.xlabel("taille graphe")
plt.ylabel("Temps d execution")

In [0]:

n=[i for i in range(50,1500,50)]
m=[1,2,3]
nb_res=[1,2,3]
b=[5,10,100]
C=[100,900,2000]
resu=[list(tup) for tup in it.product(n,m,nb_res,b,C)]
for i in range(len(resu)):
  resu[i][1]=int(resu[i][1]*resu[i][0])
df=pd.DataFrame(resu,columns = ['n', 'm', 'Ressource', 'b','Contraintes'])
df['temps']=None
df['temps_pareto']=None
df['explosion']=0

#Aboa
df[::][0:df.shape[0]:2]
#Hugo
df[::][1:df.shape[0]:2]



In [0]:
############################################
for i in range(65,df.shape[0],2):
    print("==========ITERATION========== :",i)
    temps=[]
    temps_pareto=[]
    explose=0
    for j in range(5):
      print("Numero :",j)
      G=generateur_graph(df['n'][i]-2,df['m'][i],df['Ressource'][i],b=df['b'][i])
      start_time = time.time()
      resu=correc_etiquette(G,[df['Contraintes'][i] for h in range(df['Ressource'][i])],source="s",puit="t")
      temps.append(time.time() - start_time)
      temps_pareto.append(resu[2])
      explose+=resu[1]
    print("Nombre d'explosion : ",explose)
    df['temps'][i]=np.mean(temps)
    df['temps_pareto'][i]=np.mean(temps_pareto)
    df['explosion'][i]=explose

In [0]:
print(df[::][i:i+1])
G=generateur_graph(df['n'][i]-2,df['m'][i],df['Ressource'][i],b=df['b'][i])
G.affiche()
resu=correc_etiquette(G,[df['Contraintes'][i] for h in range(df['Ressource'][i])],source="s",puit="t")

In [0]:
G=generateur_graph(1500,3*1500,3,b=10)
start_time = time.time()
resu=correc_etiquette(G,[2000,2000,2000],source="s",puit="t")
print(resu[1])
time.time() - start_time